In [1]:
import os
import pickle
import numpy as np
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import gym
from optical_rl_gym.utils import evaluate_heuristic, random_policy
# from optical_rl_gym.envs.rwa_env_focs_v2 import kSP_FF, FF_kSP, kSP_MU, CA_MU
from optical_rl_gym.heuristics import kSP_FF, FF_kSP, kSP_MU, CA_MU
from optical_rl_gym.envs.rwa_env_focs_v2 import PathOnlyFirstFitAction

/Users/joshnevin/.pyenv/versions/3.7.12/envs/rlenv37/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
def print_service_stats(env):
    print("Whole training process statistics:")
    rnd_path_action_probability = np.sum(env.actions_output, axis=1) / np.sum(env.actions_output)
    rnd_wavelength_action_probability = np.sum(env.actions_output, axis=0) / np.sum(env.actions_output)
    print('Path action probability:', np.sum(env.actions_output, axis=1) / np.sum(env.actions_output))
    print('Wavelength action probability:', np.sum(env.actions_output, axis=0) / np.sum(env.actions_output))
    num_lps_reused = env.num_lightpaths_reused
    print('Load (Erlangs):', load)
    print('Service bit rate (Gb/s):', env.service.bit_rate/1e9)
    print('Total number of services:', env.services_processed)
    print('Total number of services per ep:', env.episode_services_processed)
    print('Total number of accepted services:', env.services_accepted)
    print('Proportion of services provisioned:', env.services_accepted/env.services_processed)
    print('Number of services on existing lightpaths:', num_lps_reused)
    print('Number of services released:', env.num_lightpaths_released)
    print('Number of transmitters on each node:', env.num_transmitters)
    print('Number of receivers on each node:', env.num_receivers)
    print('Final throughput (TB/s):', env.get_throughput()/1e12)
def get_service_utils(env):
    path_id_util = []
    num_paths = env.topology.number_of_nodes() * (env.topology.number_of_nodes() - 1) * env.k_paths
    for id in range(num_paths):
        path_id_util.append(np.sum(env.lightpath_service_allocation[id,:]))
    return path_id_util, num_paths

In [3]:
# with open('/Users/joshnevin/RL_FOCSLab/topologies/3_node_network_sym.h5', 'rb') as f:
#     topology = pickle.load(f)
# node_request_probabilities = np.array([0.333333,0.333333,0.333333])

In [4]:
with open('/Users/joshnevin/RL_FOCSLab/topologies/nsfnet_chen_5-paths_directional.h5', 'rb') as f:
    topology = pickle.load(f)
# node probabilities from https://github.com/xiaoliangchenUCD/DeepRMSA/blob/6708e9a023df1ec05bfdc77804b6829e33cacfe4/Deep_RMSA_A3C.py#L77
node_request_probabilities = np.array([0.01801802, 0.04004004, 0.05305305, 0.01901902, 0.04504505,
       0.02402402, 0.06706707, 0.08908909, 0.13813814, 0.12212212,
       0.07607608, 0.12012012, 0.01901902, 0.16916917])

In [22]:
load = int(1e10)
env_args = dict(topology=topology, seed=5, load = load,
                allow_rejection=False, # the agent cannot proactively reject a request
                mean_service_holding_time=1e8,
                episode_length=1600, node_request_probabilities=node_request_probabilities, exp_request_res = 25e9,
               exp_request_lambda = 1, term_on_first_block=False)
# env_0 = PathOnlyFirstFitAction(gym.make('RWAFOCS-v2', **env_args))
env_0 = gym.make('RWAFOCS-v4', **env_args)
env_1 = gym.make('RWAFOCS-v22', **env_args)
env_2 = gym.make('RWAFOCS-v22', **env_args)
env_3 = gym.make('RWAFOCS-v22', **env_args)
env_4 = gym.make('RWAFOCS-v22', **env_args)

In [11]:
mean_reward, std_reward = evaluate_heuristic(env_0, random_policy, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)
print("1 std best case:", mean_reward + std_reward)

mean reward: -382.0
Std reward: 0.0
1 std best case: -382.0


In [12]:
mean_reward + std_reward

-382.0

In [23]:
heuristic = kSP_FF
mean_reward, std_reward = evaluate_heuristic(env_1, kSP_FF, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1354.0
Std reward: 0.0


In [24]:
mean_reward, std_reward = evaluate_heuristic(env_2, FF_kSP, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1356.0
Std reward: 0.0


In [25]:
mean_reward, std_reward = evaluate_heuristic(env_3, CA_MU, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1352.0
Std reward: 0.0


In [26]:
mean_reward, std_reward = evaluate_heuristic(env_4, kSP_MU, n_eval_episodes=1,
                       render=False, callback=None, reward_threshold=None,
                       return_episode_rewards=False)
print("mean reward:", mean_reward)
print("Std reward:", std_reward)

mean reward: 1352.0
Std reward: 0.0


kSP-FF:

In [27]:
print_service_stats(env_1)

Whole training process statistics:
Path action probability: [0.64387271 0.19363575 0.08666215 0.04129993 0.03452945]
Wavelength action probability: [0.01083277 0.00947867 0.01083277 0.01150982 0.01218687 0.01083277
 0.01218687 0.01015572 0.00947867 0.00947867 0.01015572 0.01015572
 0.01015572 0.01218687 0.01218687 0.01083277 0.01218687 0.00744753
 0.00812458 0.00880162 0.00880162 0.00947867 0.01286391 0.01150982
 0.00880162 0.00880162 0.01015572 0.00880162 0.00947867 0.01218687
 0.00880162 0.01015572 0.01083277 0.01015572 0.00744753 0.00880162
 0.01083277 0.00947867 0.01015572 0.01150982 0.00812458 0.00880162
 0.00947867 0.01218687 0.01083277 0.01015572 0.00812458 0.01015572
 0.01083277 0.00812458 0.00880162 0.00812458 0.00947867 0.01218687
 0.00812458 0.00947867 0.01150982 0.00947867 0.01150982 0.01150982
 0.00812458 0.00677048 0.00812458 0.00947867 0.00880162 0.01218687
 0.01218687 0.00947867 0.00880162 0.00947867 0.01083277 0.01150982
 0.01015572 0.00812458 0.00880162 0.01218687 0.0

FF-kSP:

In [28]:
print_service_stats(env_2)

Whole training process statistics:
Path action probability: [0.54330176 0.21853857 0.09945873 0.07983762 0.05886333]
Wavelength action probability: [0.0135318  0.01082544 0.01014885 0.01014885 0.01217862 0.01217862
 0.01082544 0.00947226 0.01082544 0.00608931 0.01082544 0.01082544
 0.01082544 0.01082544 0.01150203 0.01014885 0.01082544 0.01082544
 0.00744249 0.00744249 0.00947226 0.00811908 0.00811908 0.01150203
 0.01217862 0.01014885 0.00947226 0.00947226 0.01082544 0.01014885
 0.00947226 0.00879567 0.00947226 0.01014885 0.01217862 0.01014885
 0.01014885 0.00879567 0.01014885 0.00879567 0.00879567 0.01082544
 0.01014885 0.00947226 0.00947226 0.00947226 0.01150203 0.01082544
 0.00879567 0.00744249 0.00947226 0.00811908 0.01014885 0.0135318
 0.00879567 0.01217862 0.00879567 0.00879567 0.00947226 0.01150203
 0.0067659  0.00811908 0.00879567 0.00879567 0.01150203 0.01014885
 0.01014885 0.01150203 0.01217862 0.00947226 0.00879567 0.00947226
 0.01150203 0.01014885 0.00947226 0.01014885 0.01

CA-MU:

In [29]:
print_service_stats(env_3)

Whole training process statistics:
Path action probability: [0.91192412 0.03726287 0.01829268 0.01558266 0.01693767]
Wavelength action probability: [0.00948509 0.00948509 0.01084011 0.01084011 0.01151762 0.01151762
 0.01287263 0.01219512 0.00880759 0.00880759 0.0101626  0.0101626
 0.0101626  0.01219512 0.01084011 0.01084011 0.00948509 0.0101626
 0.00948509 0.00948509 0.00609756 0.0101626  0.0101626  0.01084011
 0.01151762 0.00745257 0.0101626  0.01355014 0.01084011 0.0101626
 0.00745257 0.00880759 0.0101626  0.00948509 0.00880759 0.00948509
 0.01287263 0.0101626  0.0101626  0.00880759 0.00745257 0.00880759
 0.01084011 0.00880759 0.0101626  0.01287263 0.01084011 0.00948509
 0.01084011 0.0101626  0.00813008 0.00745257 0.01084011 0.0101626
 0.0101626  0.0101626  0.0101626  0.0101626  0.00880759 0.00880759
 0.00745257 0.00880759 0.01084011 0.00813008 0.01084011 0.01084011
 0.01355014 0.0101626  0.00880759 0.00948509 0.01084011 0.00948509
 0.00880759 0.01084011 0.00880759 0.0101626  0.01219

kSP-MU:

In [30]:
print_service_stats(env_4)

Whole training process statistics:
Path action probability: [0.94850949 0.03116531 0.00948509 0.00609756 0.00474255]
Wavelength action probability: [0.01084011 0.01151762 0.00948509 0.01287263 0.0101626  0.01287263
 0.01151762 0.0101626  0.00948509 0.00677507 0.01219512 0.0101626
 0.01151762 0.01084011 0.01355014 0.01084011 0.00813008 0.00813008
 0.00948509 0.00745257 0.00745257 0.0101626  0.0101626  0.01151762
 0.0101626  0.00813008 0.01219512 0.0101626  0.01084011 0.00813008
 0.01084011 0.00880759 0.0101626  0.00948509 0.00880759 0.0101626
 0.01151762 0.00880759 0.01084011 0.00880759 0.0101626  0.0101626
 0.00813008 0.0101626  0.01084011 0.00948509 0.01084011 0.0101626
 0.00948509 0.00948509 0.00880759 0.01084011 0.00948509 0.01084011
 0.00948509 0.0101626  0.01084011 0.00880759 0.00677507 0.0101626
 0.00677507 0.01151762 0.00813008 0.01151762 0.01084011 0.0101626
 0.01151762 0.01151762 0.0101626  0.00880759 0.01151762 0.00880759
 0.00813008 0.0101626  0.0101626  0.01219512 0.0108401

In [ ]:
path_id_util_1, num_paths = get_service_utils(env_1)
path_id_util_2, _ = get_service_utils(env_2)
path_id_util_3, _ = get_service_utils(env_3)
path_id_util_4, _ = get_service_utils(env_4)

In [ ]:
sum(path_id_util_1)

In [ ]:
sum(path_id_util_2)

In [ ]:
sum(path_id_util_3)

In [ ]:
sum(path_id_util_4)

Verify that throughput is the same, despite the number of allocated requests being different!

In [ ]:
req_comp = np.array([env_1.cumulative_throughput[i] - env_2.cumulative_throughput[i] for i in range(len(env_1.cumulative_throughput))])

In [ ]:
np.count_nonzero(req_comp)

In [ ]:
print(req_comp[-1])
print(req_comp[-2])
print(req_comp[-3])
print(req_comp[-4])
print(req_comp[-5])
print(req_comp[-6])
print(req_comp[-7])

In [ ]:
print(env_2.cumulative_throughput[-1])
print(env_2.cumulative_throughput[-2])

In [ ]:
plt.plot(env_1.cumulative_throughput, '+')
plt.show()

In [ ]:
plt.plot(env_2.cumulative_throughput, '+')
plt.show()

In [ ]:
plt.plot(req_comp)
plt.show()

In [ ]:
np.sum(env_2.active_bitrates)

In [ ]:
env_2.cumulative_throughput

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_1)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_kspff.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_2)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_3)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_4)
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, path_id_util_1, 'r+')
plt.plot(x, path_id_util_2, 'bx')
plt.xlabel("Path ID")
plt.ylabel("Number of services")
plt.show()

In [ ]:
x = np.arange(num_paths)
plt.plot(x, abs(np.array(path_id_util_1) - np.array(path_id_util_2)))
plt.xlabel("Path ID")
plt.ylabel("Number of services")
#plt.savefig('figures/pathid_services_ffksp.pdf', dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
(np.array(path_id_util_1) - np.array(path_id_util_2))

In [ ]:
np.array(path_id_util_1)

In [ ]:
np.array(path_id_util_2)

Look at ksp ordering:

In [ ]:
env_1.service.source

In [ ]:
paths_test = env_1.topology.graph['ksp']['2','3']
len(paths_test)

In [ ]:
paths_test[0].length

In [ ]:
paths_test[1].length

In [ ]:
paths_test[2].length

In [ ]:
paths_test[3].length

In [ ]:
paths_test[4].length

In [ ]:
env_1.observation_space.sample()

In [ ]:
env_2.observation_space.sample()[:100]